<a href="https://colab.research.google.com/github/yatharthbajaj30/hate_speech_detection/blob/main/hate_speech_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from wordcloud import WordCloud 

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn import svm
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import confusion_matrix,accuracy_score

import os
for dirname, _,filenames in os.walk('/content'):
  for filename in filenames:
    print(os.path.join(dirname,filename))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/content/twitter_data.csv
/content/.config/.last_opt_in_prompt.yaml
/content/.config/.last_survey_prompt.yaml
/content/.config/.last_update_check.json
/content/.config/config_sentinel
/content/.config/active_config
/content/.config/.feature_flags_config.yaml
/content/.config/gce
/content/.config/.metricsUUID
/content/.config/configurations/config_default
/content/.config/logs/2022.04.19/14.22.38.448952.log
/content/.config/logs/2022.04.19/14.23.29.232715.log
/content/.config/logs/2022.04.19/14.22.58.550404.log
/content/.config/logs/2022.04.19/14.22.14.343189.log
/content/.config/logs/2022.04.19/14.23.07.556024.log
/content/.config/logs/2022.04.19/14.23.28.447853.log
/content/sample_data/README.md
/content/sample_data/anscombe.json
/content/sample_data/california_housing_train.csv
/content/sample_data/mnist_test.csv
/content/sample_data/california_housing_test.csv
/cont

In [ ]:
dataset=pd.read_csv('/content/twitter_data.csv')
dataset.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [ ]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,24783.0,12681.192027,7299.553863,0.0,6372.5,12703.0,18995.5,25296.0
count,24783.0,3.243473,0.883060,3.0,3.0,3.0,3.0,9.0
hate_speech,24783.0,0.280515,0.631851,0.0,0.0,0.0,0.0,7.0
offensive_language,24783.0,2.413711,1.399459,0.0,2.0,3.0,3.0,9.0
neither,24783.0,0.549247,1.113299,0.0,0.0,0.0,0.0,9.0
class,24783.0,1.110277,0.462089,0.0,1.0,1.0,1.0,2.0


In [ ]:
dt_transformed=dataset[['class','tweet']]
y=dt_transformed.iloc[:,:-1].values 

In [ ]:
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])],remainder='passthrough')
y=np.array(ct.fit_transform(y))

In [ ]:
print(y)

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
y_df=pd.DataFrame(y)
y_hate=np.array(y_df[0])
y_offensive=np.array(y_df[1])

In [ ]:
print(y_hate)
print(y_offensive)

[0. 0. 0. ... 0. 0. 0.]
[0. 1. 1. ... 1. 1. 0.]


In [ ]:
corpus=[]
for i in range(0,24783):
  review=re.sub('[^a-zA-Z]', ' ',dt_transformed['tweet'][i])
  review=review.lower()
  review=review.split()
  ps=PorterStemmer()
  all_stopwords=stopwords.words('english')
  all_stopwords.remove('not')
  review=[ps.stem(word) for word in review if not word in set(all_stopwords)]
  review=' '.join(review)
  corpus.append(review)


In [ ]:
cv=CountVectorizer(max_features=2000)
x=cv.fit_transform(corpus).toarray()

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y_hate,test_size=0.30,random_state=0)

In [ ]:
classifier_np=GaussianNB()
classifier_np.fit(x_train,y_train)

GaussianNB()

In [ ]:
classifier_lr=LogisticRegression(random_state=0)
classifier_lr.fit(x_train,y_train)

LogisticRegression(random_state=0)

In [ ]:
classifier_svm=svm.SVC()
classifier_svm.fit(x_train,y_train)

SVC()

In [ ]:
y_pred_np=classifier_np.predict(x_test)
cm=confusion_matrix(y_test,y_pred_np)
print(cm)

[[3289 3719]
 [ 168  259]]


In [ ]:
y_pred_svm=classifier_svm.predict(x_test)
cm=confusion_matrix(y_test,y_pred_svm)
print(cm)

[[6964   44]
 [ 372   55]]


In [ ]:
y_pred_lr=classifier_lr.predict(x_test)
cm=confusion_matrix(y_test,y_pred_lr)
print(cm)

[[6911   97]
 [ 347   80]]


In [ ]:
svm_score=accuracy_score(y_test,y_pred_svm)
lr_score=accuracy_score(y_test,y_pred_lr)
np_score=accuracy_score(y_test,y_pred_np)

In [ ]:
print('Support vector machine accuracy: ',str(svm_score))
print('logistic Regression accuracy: ',str(lr_score))
print('naive Bayes accuracy: ',str(np_score))

Support vector machine accuracy:  0.9440484196368527
logistic Regression accuracy:  0.9402824478816408
naive Bayes accuracy:  0.4772024209818426
